In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift
from kmodes.kprototypes import KPrototypes

In [ ]:
reviews = pd.read_csv(".//CSV'S/Using/reviews.csv", error_bad_lines=False)
reviews_bis = pd.read_csv(".//CSV'S/Using/reviews_bis.csv", error_bad_lines=False)

# Join the dataframes based on the specified conditions
reviews_merged = pd.merge(reviews, reviews_bis, on ='review_id', how='left')
import string
from textblob import TextBlob

def preprocess_text(text):
    if text is None or not isinstance(text, str):
        return ""
    # lowercase text
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def get_sentiment(text):
    if not isinstance(text, str) or text == "":
        return 'neutral'
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

reviews_merged['review_comment_message'] = reviews_merged['review_comment_message'].apply(preprocess_text)
reviews_merged['sentiment'] = reviews_merged['review_comment_message'].apply(get_sentiment)

In [ ]:
# Load the data from CSV files into pandas dataframes
result = pd.read_excel(".//CSV'S/Using/result_wip_cleaned.xlsx")

In [ ]:
# Join the dataframes based on the specified conditions
result = pd.merge(result, reviews_merged[['order_id_x', 'sentiment']], left_on='order_id', right_on='order_id_x', how='left')

# Drop the duplicate column
result.drop('order_id_x', axis=1, inplace=True)

In [ ]:
result.info()

In [ ]:
# Count the number of missing values per column
num_missing = result.isna().sum()

# Display the number of missing values per column
print("Number of missing values per column:")
print(num_missing)

In [ ]:
# Fill lead_type with the most frequent value
most_frequent_lead_type = result['lead_type'].mode()[0]
result['lead_type'] = result['lead_type'].fillna(most_frequent_lead_type)

# replace NA values in the "origin" column with "Other"
result['origin'].fillna('other', inplace=True)

# replace "Unknown" values in the "origin" column with "Other"
result['origin'].replace('unknown', 'other', inplace=True)

# Fill lead_behaviour with the most frequent value
most_frequent_lead_behavior = result['lead_behavior'].mode()[0]
result['lead_behavior'] = result['lead_behavior'].fillna(most_frequent_lead_behavior)

# Fill business_type with the most frequent value
most_frequent_business_type = result['business_type'].mode()[0]
result['business_type'] = result['business_type'].fillna(most_frequent_business_type)

# Fill won_date_ie_date_of_first_saas_payment with median
won_date_median = result['won_date_ie_date_of_first_saas_payment'].median()
result['won_date_ie_date_of_first_saas_payment'] = result['won_date_ie_date_of_first_saas_payment'].fillna(won_date_median)

# Fill order_date_time with median
order_date_median = result['order_date_time'].median()
result['order_date_time'] = result['order_date_time'].fillna(order_date_median)

# Fill lead_type with the most frequent value
most_frequent_sentiment_type = result['sentiment'].mode()[0]
result['sentiment'] = result['sentiment'].fillna(most_frequent_sentiment_type)


In [ ]:
# Count the number of missing values per column
num_missing = result.isna().sum()

# Display the number of missing values per column
print("Number of missing values per column:")
print(num_missing)

In [ ]:
# export dataframe to excel
# result.to_excel('result_final_1.xlsx', index=False)

In [ ]:
unique_values = result.nunique()
print(unique_values)

In [ ]:
# convert columns to datetime format
result['registered_on_landing_page_date'] = pd.to_datetime(result['registered_on_landing_page_date'])
result['won_date_ie_date_of_first_saas_payment'] = pd.to_datetime(result['won_date_ie_date_of_first_saas_payment'])

# calculate the turnaround time in days
result['turnaround_time'] = (result['won_date_ie_date_of_first_saas_payment'] - result['registered_on_landing_page_date']).dt.days

# count the number of dates
num_dates = result['turnaround_time'].count()


In [ ]:
# convert columns to datetime format
result['won_date_ie_date_of_first_saas_payment'] = pd.to_datetime(result['won_date_ie_date_of_first_saas_payment'])
result['order_date_time'] = pd.to_datetime(result['order_date_time'])

# calculate the turnaround time in days
result['first_order_tat'] = (result['order_date_time'] - result['won_date_ie_date_of_first_saas_payment']).dt.days

# count the number of dates
num_dates = result['first_order_tat'].count()

# print the number of dates
print(f"Number of dates: {num_dates}")


In [ ]:
# group by business segment and calculate the average turnaround time
avg_tat_by_segment = result.groupby('business_segment')['turnaround_time'].mean()

# sort by ascending average turnaround time
avg_tat_by_segment = avg_tat_by_segment.sort_values()

# create a horizontal bar chart with adjusted spacing
fig, ax = plt.subplots(figsize=(12,8))
ax.barh(avg_tat_by_segment.index, avg_tat_by_segment.values, height=0.6)

# add labels and title
ax.set_ylabel('Business Segment')
ax.set_xlabel('Average Turnaround Time')
ax.set_title('Average Turnaround Time by Business Segment')

# display the plot
plt.show()


In [ ]:
# count the unique values of sr_id, sdr_id, product_id, and merchant_id
unique_sr_id = result['sr_id'].nunique()
unique_sdr_id = result['sdr_id'].nunique()
unique_product_id = result['product_id'].nunique()
unique_merchant_id = result['merchant_id'].nunique()

# print the counts
print("Number of unique sr_id values:", unique_sr_id)
print("Number of unique sdr_id values:", unique_sdr_id)
print("Number of unique product_id values:", unique_product_id)
print("Number of unique merchant_id values:", unique_merchant_id)

In [ ]:
# get the unique sr_id and sdr_id values
unique_sr_id = result['sr_id'].unique()
unique_sdr_id = result['sdr_id'].unique()

# print the number of unique sr_id and sdr_id values
print("Number of unique sr_id values:", len(unique_sr_id))
print("Number of unique sdr_id values:", len(unique_sdr_id))

In [ ]:
# group the data by merchant_id and count the number of unique product_id values for each group
product_count_by_merchant_id = result.groupby('merchant_id')['product_id'].nunique().sort_values(ascending=False)

# print the resulting Series
print(product_count_by_merchant_id)

# calculate the total number of unique products ordered by all merchants
total_unique_products = product_count_by_merchant_id.sum()

print("Total unique products ordered by all merchants:", total_unique_products)


In [ ]:
# assuming your DataFrame is called 'df' and has columns 'merchant_id' and 'product_id'
unique_merchant_ids = result['merchant_id'].unique()
total_product_count = 0
for merchant_id in unique_merchant_ids:
    product_count = result[result['merchant_id'] == merchant_id]['product_id'].count()
    total_product_count += product_count
    print(f"Merchant ID: {merchant_id} - Product count: {product_count}")
print(f"Total product count: {total_product_count}")


In [ ]:
result.info()

In [ ]:
unique_merchant_ids = result['merchant_id'].unique()
unique_merchant_ids

In [ ]:
# read data from a csv file
df = result.copy()

# group by business segment and count unique product ids
product_count = df.groupby('business_segment')['product_id'].nunique()

# sort values in descending order
product_count = product_count.sort_values(ascending=False)

# create a horizontal bar chart
product_count.plot(kind='barh')
plt.xlabel('Number of Products')
plt.ylabel('Business Segment')
plt.title('Number of Products Sold in Each Business Segment')
plt.show()


In [ ]:
# read data from a csv file
df = result.copy()

# extract the month from the order date
df['month'] = df['order_date_time'].dt.month_name()

# define the order of the months
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# convert the month column to a categorical data type with the specified order
df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)

# group by business segment, month, and product id and count unique product ids
product_count = df.groupby(['business_segment', 'month'])['product_id'].nunique().reset_index()

# create a pivot table to reshape the data
product_count_pivot = product_count.pivot(index='business_segment', columns='month', values='product_id')

# increase the size of the heatmap
fig, ax = plt.subplots(figsize=(10, 8))

# create a heatmap to visualize the data
im = ax.imshow(product_count_pivot, cmap='Blues')

# set x-tick labels and rotation
ax.set_xticks(range(len(product_count_pivot.columns)))
ax.set_xticklabels(product_count_pivot.columns, rotation=45, ha='right')

# set y-tick labels
ax.set_yticks(range(len(product_count_pivot.index)))
ax.set_yticklabels(product_count_pivot.index)

# add x and y axis labels and title
plt.xlabel('Month')
plt.ylabel('Business Segment')
plt.title('Number of Products Sold by Segment and Month')

# add colorbar
plt.colorbar(im)

# display the heatmap
plt.show()


In [ ]:
result_dummies = result[['lead_id', 'registered_on_landing_page_date', 'lead_type', 'origin', 'business_segment', 'lead_behavior', 'has_company', 'has_gtin', 'business_type', 'won_date_ie_date_of_first_saas_payment', 'order_line_item', 'order_date_time', 'price', 'freight_cost', 'turnaround_time', 'first_order_tat','sentiment']]

In [ ]:
# Select the columns for which you want to compute the correlation matrix
cols = ['registered_on_landing_page_date', 'has_company', 'has_gtin', 'price', 'freight_cost', 'turnaround_time', 'first_order_tat','sentiment']

# Compute the correlation matrix
corr_matrix = result_dummies[cols].corr()

# Plot a heatmap of the correlation matrix
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, fmt='.2f')

# Show the plot
plt.show()

In [ ]:
result_dummies.info()

In [ ]:

# create dummy variables for the columns
dummy_cols = ['lead_type', 'origin', 'business_segment', 'lead_behavior', 'business_type','sentiment']
result_dummies = pd.get_dummies(result_dummies, columns=dummy_cols)

# Map True and False values to 1 and 0
result_dummies['has_company'] = result_dummies['has_company'].map({True: 1, False: 0})
result_dummies['has_gtin'] = result_dummies['has_gtin'].map({True: 1, False: 0})

# Extract month and year from 'registered_on_landing_page_date'
result_dummies['registered_on_landing_page_month'] = pd.to_datetime(result_dummies['registered_on_landing_page_date']).dt.month
result_dummies['registered_on_landing_page_year'] = pd.to_datetime(result_dummies['registered_on_landing_page_date']).dt.year

# Extract month and year from 'won_date_ie_date_of_first_saas_payment'
result_dummies['won_date_month'] = pd.to_datetime(result_dummies['won_date_ie_date_of_first_saas_payment']).dt.month
result_dummies['won_date_year'] = pd.to_datetime(result_dummies['won_date_ie_date_of_first_saas_payment']).dt.year

# Extract month and year from 'order_date_time'
result_dummies['order_date_month'] = pd.to_datetime(result_dummies['order_date_time']).dt.month
result_dummies['order_date_year'] = pd.to_datetime(result_dummies['order_date_time']).dt.year

# Drop original columns
result_dummies = result_dummies.drop(['registered_on_landing_page_date', 'won_date_ie_date_of_first_saas_payment', 'order_date_time','lead_id'], axis=1)

result_dummies.reset_index()


In [ ]:
result_dummies.info()

In [ ]:
# Split data into train and test sets
train_data, test_data = train_test_split(result_dummies, test_size=0.30, random_state=100)

# Define the number of clusters
n_clusters = 3

# Instantiate the models
models = [
    KMeans(n_clusters=n_clusters, n_init=10, random_state=42),
    AgglomerativeClustering(n_clusters=n_clusters),
    DBSCAN(),
    KPrototypes(n_clusters=n_clusters, init='Cao', verbose=0)
]

# Train and evaluate the models
for model in models:
    if isinstance(model, (AgglomerativeClustering, DBSCAN)):
        test_labels = model.fit_predict(test_data)
    elif isinstance(model, KPrototypes):
        model.fit(train_data, categorical=list(range(6)))
        test_labels = model.predict(test_data, categorical=list(range(6)))
        print(f"{type(model).__name__} Predicted Labels: {test_labels}")
    else:
        model.fit(train_data)
        test_labels = model.predict(test_data)
    
    silhouette_avg = silhouette_score(test_data, test_labels)
    
    # Print the silhouette score for the model
    print(f"{type(model).__name__} Silhouette Score: {silhouette_avg:.2f}")